In [ ]:
import pandas as pd

def assim_que_eu_fiz_p_organizar():
  for i in range(2014,2024):
    j = str(i)
    nome_arquivo = "relatorio_preco_medio_mensal_"+j+".csv"

    # Abrir o arquivo CSV
    df = pd.read_csv(nome_arquivo)

    # Preencher campos vazios com o valor anterior
    coluna_preenchimento = 'Produto/Unidade'
    df[coluna_preenchimento] = df[coluna_preenchimento].fillna(method='ffill')

    # Preencher campos vazios com o valor anterior
    coluna_preenchimento = 'Nível de Comercialização'
    df[coluna_preenchimento] = df[coluna_preenchimento].fillna(method='ffill')

    # Ordenar as linhas com base em uma coluna específica
    coluna_ordenacao = 'U.F.'
    df = df.sort_values(by=coluna_ordenacao)
    # Ordenar as linhas com base em mais de uma coluna com direção de ordenação especificada
    colunas_ordenacao = ['U.F.','Nível de Comercialização','Produto/Unidade']
    df = df.sort_values(by=colunas_ordenacao)
    # Escrever o DataFrame de volta para um arquivo CSV
    df.to_csv("Dados_vendas_organizado/"+nome_arquivo, index=False)




# Exibir o DataFrame resultante
#print(df)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import glob
import sys
sys.path.append('/content/drive/MyDrive/ideias_de_projeto')


# Lista para armazenar os DataFrames
dfs = []

for i in range(2019,2024):
  j = str(i)
  nome_arquivo = "/content/drive/MyDrive/Ideias_de_projeto/Precos_mensais_anuias_dataset/relatorio_preco_medio_mensal_"+j+".csv"
  # Abrir o arquivo CSV
  df = pd.read_csv(nome_arquivo)
  print(df)
  dfs.append(df)

# Iniciar com o primeiro DataFrame
df_final = dfs[0]

# Iterar sobre os DataFrames restantes e fazer a junção
for df in dfs[1:]:
    df_final = pd.merge(df_final, df, on=['Produto/Unidade', 'Nível de Comercialização', 'U.F.'], how='outer')

print(df_final)



df_varejo = df_final[df_final['Nível de Comercialização'] == 'VAREJO']
df_atacado = df_final[df_final['Nível de Comercialização'] == 'ATACADO']
df_produtor = df_final[df_final['Nível de Comercialização'] == 'PRODUTOR']


                                        Produto/Unidade  \
0           ARROZ LONGO FINO BENEFICIADO TIPO 1 (25 kg)   
1                           CAFÉ MOÍDO E TORRADO (5 kg)   
2          CARNE BOVINA DIANTEIRO COM OSSO (25 - 30 kg)   
3                     CARNE DE FRANGO CONGELADO (20 kg)   
4         CARNE SUÍNA CONGELADO PERNIL COM OSSO (12 kg)   
...                                                 ...   
2083  As informações podem ser reproduzidas, desde q...   
2084  Fonte: Companhia Nacional de Abastecimento - C...   
2085  Proibida a comercialização das informações dis...   
2086                                 VACA GORDA (15 kg)   
2087                                 VACA GORDA (15 kg)   

     Nível de Comercialização U.F. 01/2019 02/2019 03/2019 04/2019 05/2019  \
0                     ATACADO   AC   67,24   67,15   66,72   66,79   66,58   
1                     ATACADO   AC   93,80   94,06   94,19   93,91   94,21   
2                     ATACADO   AC  223,65  224,30  224,7

In [ ]:
import pandas as pd

In [ ]:
df_varejo = pd.read_csv('/content/drive/MyDrive/Ideias_de_projeto/Precos_por_Nivel_de_comercializacao/dados_varejo_consolidados.csv')
df_atacado = pd.read_csv('/content/drive/MyDrive/Ideias_de_projeto/Precos_por_Nivel_de_comercializacao/dados_atacado_consolidados.csv')
df_produtor = pd.read_csv('/content/drive/MyDrive/Ideias_de_projeto/Precos_por_Nivel_de_comercializacao/dados_produtor_consolidados.csv')


In [ ]:
df = df_varejo.drop(columns=['Nível de Comercialização'])
df = df.drop(columns=['Produto/Unidade'])
_# Substituir os valores NaN pela interpolação
df.iloc[:, 1:] = df.iloc[:, 1:].apply(lambda x: x.interpolate(method='nearest', axis=0, limit_direction='both'))


df.iloc[:, 1:] = df.iloc[:, 1:].apply(lambda x: x.str.replace(',', '.'))
# Converter todos os valores para string
df = df.astype(str)
df.iloc[:, 1:] = df.iloc[:, 1:].apply(lambda col: col.apply(lambda x: x.replace('.', '', 1) if x.count('.') == 2 else x))
df.iloc[:, 1:] = df.iloc[:, 1:].apply(lambda col: col.str.replace(',', '.').astype(float))


# 537 -> varejo
df = df[:537] # valores mudam conforme tipo
df = df.T

# Preencher valores nulos primeiro com forward fill e depois com backward fill
df_combined = df.fillna(method='ffill').fillna(method='bfill')

print(df_combined)

          0     1      2     3     4     5     6     7     8     9    ...  \
U.F.       AC    AC     AC    AC    AC    AC    AC    AC    AC    AC  ...   
01/2019  2.82  2.57  12.95  6.15  2.35   3.6   4.0  3.17  3.75  5.15  ...   
02/2019  2.75  2.55  12.95   6.0  2.25   3.5  3.95  3.12   3.5   5.2  ...   
03/2019  2.77  2.55  12.97   6.0   2.3  3.55  4.03  3.15  3.45  5.22  ...   
04/2019  2.77  2.55   13.1   6.2  2.45  3.45  4.25   3.2  3.55  5.35  ...   
...       ...   ...    ...   ...   ...   ...   ...   ...   ...   ...  ...   
08/2023   5.1  3.05   28.0  9.23  4.77   5.2  7.22  4.45   7.0  7.55  ...   
09/2023   5.1  3.05   28.0  9.23  4.77  5.13  7.22  4.45   7.0  7.55  ...   
10/2023  5.14  3.09   28.0  9.23  4.77  5.12  7.27  4.45  6.85  7.51  ...   
11/2023  5.15   3.1   28.0  9.23  4.77  5.12  7.28  4.45   6.8  7.52  ...   
12/2023  5.15   3.1   28.0  9.23  4.77  5.12  7.24  4.45   6.8  7.55  ...   

          527   528    529   530   531   532   533    534   535   536  
U.F

In [ ]:
# Dicionário de substituição
substituicao = ['AC','AL','AP','AM','BA','CE','DF','ES','GO','MA','MT','MS','MG','PA','PB','PR','PE','PI','RJ','RN','RS','RO','RR','SC','SP','SE','TO']
print(substituicao )
df_combined = df_combined.T

for tome in substituicao:

  df_aux = df_combined[df_combined['U.F.'] == tome];

  df_aux = df_aux.drop(columns=['U.F.'])
  df_aux = df_aux.T

  print(df_aux)
  onde_quarda = '/content/drive/MyDrive/Ideias_de_projeto/Precos_por_Nivel_de_comercializacao/Varejo/'+ tome +'_v.csv'
  df_aux = df_aux.to_csv(onde_quarda, index=False)


['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']
           0     1      2     3     4     5     6     7     8     9      10  \
01/2019  2.82  2.57  12.95  6.15  2.35   3.6   4.0  3.17  3.75  5.15    8.9   
02/2019  2.75  2.55  12.95   6.0  2.25   3.5  3.95  3.12   3.5   5.2    8.9   
03/2019  2.77  2.55  12.97   6.0   2.3  3.55  4.03  3.15  3.45  5.22    8.9   
04/2019  2.77  2.55   13.1   6.2  2.45  3.45  4.25   3.2  3.55  5.35   9.15   
05/2019  2.78   2.5   13.0  6.22  2.55   3.5   4.4   3.1   3.6   5.5   10.0   
06/2019   2.8  2.52   13.0   6.5  2.65  3.52   4.4   3.2  3.55  5.55  10.25   
07/2019   2.8  2.52   13.0   6.5   2.7  3.52   4.4   3.3   3.6  5.55  10.35   
08/2019  2.85  2.55   13.1  6.55   2.7  3.52  4.43  3.33  3.62   5.6  10.37   
09/2019  2.87   2.5  13.15  6.55  2.85  3.55   4.5  3.35  3.65  5.65   10.4   
10/2019  2.87  2.48   13.1   6.6  2.95  3.55  4

In [125]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
# Suponha que 'dados' seja seu DataFrame com datas como índices e valores de inflação para cada produto em cada mês
dados = pd.read_csv('/content/drive/MyDrive/Ideias_de_projeto/Precos_por_Nivel_de_comercializacao/Varejo/SP_v.csv')
# Calcular a média das inflações de todos os produtos para cada mês
print(dados)
preco_media = dados.T.sum(axis=0)
print(preco_media)
print(preco_media.pct_change(axis='rows') * 100)
#inflacao_media = inflacao_media.pct_change(axis='rows') * 100

preco_media = preco_media.iloc[1:]
#print(inflacao_media)
# Criar uma lista dupla com o número do mês e o número do ano
meses_anos = []
tometome = 0
for i in range(0,59):
  tometome = tometome + 30;
  meses_anos.append([int(tometome)])

# Converter para numpy array
meses_anos = np.array(meses_anos)

######### tratar outliers ######


########## normalizar ############
from sklearn.preprocessing import StandardScaler

X = np.array(meses_anos)
scaler = StandardScaler()
y = scaler.fit_transform(np.array(preco_media).reshape(-1, 1))
print(y)
###################################################

############# separar em dados de treino e teste########################3

from sklearn.model_selection import train_test_split

# Dividir os dados em conjuntos de treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=100)

############## treinar #####################
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Definir a validação cruzada com 5 dobras
cv = KFold(n_splits=5, shuffle=True, random_state=100)

# Treinar o modelo de regressão linear
lin_reg = LinearRegression().fit(X_treino, y_treino)

# Fazer previsões no conjunto de treinamento
X_predito = lin_reg.predict(X_treino)

# Fazer previsões no conjunto de teste
y_predito = lin_reg.predict( X_teste)

################# AVALIAR #################

lr_score_test = mean_squared_error(y_teste, y_predito, squared=False)
print("Pontuação RMSE: {:.2f}%".format(round((lr_score_test*100),2)))

rmse_scores = np.sqrt(-cross_val_score(lin_reg, X_teste, y_teste, cv=cv, scoring='neg_mean_squared_error'))
print("Pontuação RMSE com CV: {:.2f}%".format(round((np.mean(rmse_scores)*100),2)))

# Get R2 score
test_scores = lin_reg.score(X_teste, y_teste)
print("Pontuação R²: {:.2f}%".format(round((np.mean(test_scores)*100),2)))

# R2 and CV
score_r2_cv = cross_val_score(lin_reg, X_teste, y_teste, cv=cv, scoring='r2')
print("Pontuação R² com CV: {:.2f}%".format(round((np.mean(score_r2_cv)*100),2)))

from sklearn.metrics import r2_score

# Suponha que y_teste contenha os valores reais e y_predito contenha os valores previstos pelo modelo

# Calcula o R²
r2 = r2_score(y_teste, y_predito)
print(r2)

meses_anos_news = [ [1820] ,[1850], [1880], [1910]]
meses_anos_news = np.array(meses_anos_news)

previsao = lin_reg.predict(meses_anos_news)
print(previsao)
preco_media_original = scaler.inverse_transform(previsao)
print(preco_media_original)
# Inflação desacelera a 0,16% em março e fica abaixo das projeções. IPCA também perde força no acumulado de 12 meses e atinge 3,93%, diz IBGE.
# -0.26551676
# -0.26580587
# -0.26609498


      489   490   491   492    493    494    495    496    497   498  ...  \
0   16.70  2.29  3.34  3.34   9.41  15.38  13.89   5.99  12.18  3.29  ...   
1   15.92  2.19  4.74  4.74   9.46  15.14  15.23   5.99  11.77  3.64  ...   
2   16.40  2.25  4.79  4.79   9.65  14.97  14.79   6.39  10.99  3.73  ...   
3   15.85  1.98  5.96  5.96   9.50  14.18  12.99   6.34  12.90  3.96  ...   
4   16.14  1.99  5.99  5.99   9.34  14.88  13.90   6.69  13.10  3.49  ...   
5   16.32  1.99  5.81  5.81   9.21  14.99  13.90   6.55  13.10  3.93  ...   
6   17.44  2.12  5.74  5.74   8.98  15.52  11.24   6.56  13.10  3.96  ...   
7   15.76  2.39  4.98  4.98   9.66  16.45  13.99   7.44  11.59  4.02  ...   
8   14.53  2.29  4.59  4.59   9.58  16.70  13.55   7.47  11.85  3.73  ...   
9   16.75  2.54  3.09  3.09   9.44  15.03  12.25   6.09  11.59  4.04  ...   
10  16.45  2.29  3.19  3.19   8.77  15.47  13.35   6.69  12.71  4.34  ...   
11  17.75  2.57  4.24  4.24   9.36  17.01  11.85   6.99  14.39  4.44  ...   

In [126]:
# junho
# julho
# agosto
# setembro

infla1 = ( (278.22371285 - 276.39066762)/276.39066762 ) * 100
infla2 = ( (280.05675808 - 278.22371285 )/278.22371285 ) * 100
infla3 = ( (281.88980332 - 280.05675808)/280.05675808 ) * 100
print(infla1)
print(infla2)
print(infla3)

0.6632080763740654
0.6588386055318871
0.6545263369350183
